# Widgets Overview

The following is a run through of the widgets you can create with Interact. All widgets allow for the following keyword arguments:

* `label`: Label to be shown next to the widget
* `value`: The value the widget should be set to when created
* `signal`: A signal object of type `Reactive.Input` which gets the value of the widget as user enters input.

Many of the widgets have keyword arguments specific to them. See below for more.

In [1]:
using Interact
using Reactive

## Slider

Sliders are arguably the most useful of the widgets. A slider can be created with the `slider{T <: Number}(range::Range{T})` function. The value of the slider defaults to the median of the range, and can be set using the `value::T` keyword argument. The type of signal a slider depends on the type of the range. E.g. A floating point range like `0:π/4:2π` gives a signal of floating point values, while a range like `1:10` gives a signal of integers.

In [2]:
float_slider = slider(0:π/4:2π)

Slider{Float64}([Input{Float64}] 3.141592653589793,"",3.141592653589793,0.0:0.7853981633974483:6.283185307179586)

In [3]:
int_slider = slider(1:10)

Slider{Int64}([Input{Int64}] 5,"",5,1:10)

In [4]:
signal(int_slider)

5

## Checkbox

`checkbox` takes an optional first argument which defaults to `false` and creates a checkbox.

In [5]:
display(checkbox())
checkbox(true)

Checkbox([Input{Bool}] false,"",false)

Checkbox([Input{Bool}] true,"",true)

## Toggle

You can create a toggle button with `togglebutton` which takes as an optional argument its label.

In [6]:
status = togglebutton("Mary called", value=true)

ToggleButton([Input{Bool}] true,"Mary called",true)

In [7]:
lift(s -> s ? "Mary called" : "Mary didn't call", status)

"Mary called"

## Button

A button gives out a signal of a constant signal which is `nothing` by default. You can set this using the `value` keyword argument. The signal updates when the button is clicked. 

In [8]:
b = button("Click Me")

Button{Nothing}([Input{Nothing}] nothing,"Click Me",nothing)

Here is how you can count the number of clicks made on a button using `foldl` on the signal:

In [9]:
foldl((acc, value) -> acc + 1, 0, signal(b))

0

## Option widgets

There are 3 options widgets: `dropdown`, `togglebuttons`, `radiobuttons`. There are two types allowed as an argument while invoking these:
1. `AbstractArray` (e.g. `Vector`, `Tuple`)
2. `Associative` (e.g. `Dict`, `OrderedDict`)
The default value is the first element (or undefined in case of an undordered `Associative` like `Dict`), but this can be set using the `value` keyword argument.

In [10]:
a = dropdown(["one", "two", "three"])

Options{:Dropdown,ASCIIString}([Input{ASCIIString}] one,"","one","one",OptionDict({"one","two","three"},{"two"=>"two","one"=>"one","three"=>"three"}),None[],None[])

In [11]:
signal(a)

"one"

In [12]:
f = radiobuttons(["Add" => +, "Sub" => -, "Exp" => ^])

Options{:RadioButtons,Function}([Input{Function}] -,"",-,"Sub",OptionDict({"Sub","Add","Exp"},{"Sub"=>-,"Add"=>+,"Exp"=>^}),None[],None[])

In [13]:
@lift f(e, π*im)

2.718281828459045 - 3.141592653589793im

Notice that the order "Add", "Sub", "Exp" was not retained in the above example, because a `Dict` does not save the ordering. To overcome this, we can use `OrderedDict` from DataStructures.jl package.

In [14]:
using DataStructures
f_ = togglebuttons([("Add", +), ("Sub", -), ("Exp", ^)])

Options{:ToggleButtons,Function}([Input{Function}] +,"",+,"Add",OptionDict({"Add","Sub","Exp"},{"Sub"=>-,"Add"=>+,"Exp"=>^}),None[],None[])

In [15]:
@lift f_(e, π*im)

2.718281828459045 + 3.141592653589793im

## Textbox

A textbox can be of a `Number` or `String` type. `textbox` takes one argument: its default value.

In [16]:
string_box = textbox("Change me")

Textbox{UTF8String}([Input{UTF8String}] Change me,"",nothing,"Change me")

In [17]:
signal(string_box)

"Change me"

A textbox can be of a `Number` type as well. Just set a default number value, or use `textbox(typ=T)` where T is a `Number` type.

In [18]:
int_box = textbox(0)

Textbox{Int64}([Input{Int64}] 0,"",nothing,0)

In [19]:
signal(int_box)

0

If creating a number typed textbox, you can also pass along an optional `range` field to set a bound on the possible values one can input. If an entered value exceeds the range, it is replaced by its nearest bounding number.

In [20]:
bounded_float_box = textbox(2pi, range=-10.0:10)

Textbox{Float64}([Input{Float64}] 6.283185307179586,"",-10.0:1.0:10.0,6.283185307179586)

In [21]:
signal(bounded_float_box)

6.283185307179586

## Textarea

`textarea` takes an optional default value and creates a textarea. Its signal changes when you type.

In [22]:
tex = textarea("Your very own \$\\LaTeX\$ editor")

Textarea{ASCIIString}([Input{ASCIIString}] Your very own $\LaTeX$ editor,"","Your very own \$\\LaTeX\$ editor")

In [23]:
@lift latex(tex)

Latex("","Your very own \$\\LaTeX\$ editor")

## The `widget` Function

`widget` tries to coerce a value into a widget.

In [24]:
map(display, [
    widget(1:10),                 # Slider
    widget(false),                # Checkbox
    widget("text"),               # Textbox
    widget(1.1),                  # Number Textbox
    widget([:on, :off]),          # Toggle Buttons
    widget([:π => float(π), :τ => 2π])
    ]);

Slider{Int64}([Input{Int64}] 5,"",5,1:10)

Checkbox([Input{Bool}] false,"",false)

Textbox{String}([Input{String}] text,"",nothing,"text")

Textbox{Float64}([Input{Float64}] 1.1,"",nothing,1.1)

Options{:ToggleButtons,Symbol}([Input{Symbol}] on,"",:on,"on",OptionDict({"on","off"},{"off"=>:off,"on"=>:on}),None[],None[])

Options{:ToggleButtons,Float64}([Input{Float64}] 6.283185307179586,"",6.283185307179586,"τ",OptionDict({"τ","π"},{"τ"=>6.283185307179586,"π"=>3.141592653589793}),None[],None[])